In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
import numpy as np

Using Theano backend.


In [2]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [3]:
model = VGG_16('vgg16_weights_th_dim_ordering_th_kernels.h5')
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy')

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '9404' (I am process '9076')


ImportError: ('The following error happened while compiling the node', DeepCopyOp(convolution2d_1_W), '\n', 'DLL load failed: The specified procedure could not be found.', '[DeepCopyOp(convolution2d_1_W)]')

In [7]:
model.layers.pop()
model.layers.pop()
model.layers.pop()

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [9]:
#adam = Adam(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='Adam', loss='categorical_crossentropy')

In [10]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_2 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_2[0][0]            
____________________________________________________________________________________________________
zeropadding2d_3 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_3[0][0]            
___________________________________________________________________________________________

In [11]:
import PIL
from PIL import Image
size = 224
def read_image(f):
    im = Image.open(f)
    im = im.resize((size, size), PIL.Image.NEAREST)
    im = np.asarray(im, dtype='float64')
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    im = im.reshape(3,224,224)
    
    return(im)

In [12]:
im = read_image('1.jpg')
print(im.shape)

(3L, 224L, 224L)


In [17]:
import os
img_files = []
for root, dirs, files in os.walk('../train_samples'):
    img_files.extend(files)

In [18]:
print(len(files))

4000


In [19]:
sample_size = 100
sample = [img_files[i] for i in np.random.choice(len(files),size=sample_size, replace=False)]

In [20]:
i=0
train=[]
y_train = []
for file in sample:
    im = read_image('../train_samples/'+file)
    train.append(im)
    y_train.extend([0 if file[:3]=='cat' else 1])
    i+=1

In [22]:
x_train = np.array(train)
print(x_train.shape)

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
print(y_train.shape)

(100L, 3L, 224L, 224L)
(100L, 2L)


In [ ]:
model.fit(x_train, y_train, nb_epoch=10)

Epoch 1/10
100/100 [==============================] - 444s - loss: 0.6904    
Epoch 2/10
 32/100 [========>.....................] - ETA: 218s - loss: 0.6946

Exception 